In [145]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [146]:
questions = [
    {
        "js":[
                "How to install nodejs",
                "How do I install ts",
                "Why was ts created",
                "my javascript code is not working can some help me"
        ]
    },
    {
        "python":[
                "How to create a virtual environment",
                "my pip is not working can someone help me",
                "how do upgrade my pip",
                "I want to update my python to verion 2.4"
        ]
    }
    
]

stop_words = set(stopwords.words('english'))

In [147]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return " ".join(filtered_tokens)

In [148]:
all_questions = []
all_labels = []

In [149]:
tokenizer = Tokenizer()

In [150]:
for item in questions:
    for tag, tag_questions in item.items():
        # Preprocess all questions for the current tag
        preprocessed_questions = [preprocess_text(question) for question in tag_questions]
        
        # Append the preprocessed questions and corresponding labels
        all_questions.extend(preprocessed_questions)
        all_labels.extend([tag] * len(tag_questions))

In [151]:
tokenizer.fit_on_texts(all_questions)
sequences = tokenizer.texts_to_sequences(all_questions)

In [152]:
max_sequence_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [153]:
label_to_index = {tag: i for i, tag in enumerate(set(all_labels))}
labels = np.array([label_to_index[label] for label in all_labels], dtype=np.int32)

In [154]:
model = tf.keras.Sequential()

In [155]:
model.add(tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1,output_dim=100, input_length=max_sequence_length ))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(len(tags),activation='softmax'))

In [156]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 4, 100)            1900      
                                                                 
 flatten_9 (Flatten)         (None, 400)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                25664     
                                                                 
 dense_19 (Dense)            (None, 1)                 65        
                                                                 
Total params: 27,629
Trainable params: 27,629
Non-trainable params: 0
_________________________________________________________________


In [157]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [158]:
model.fit(sequences, labels, epochs=100, batch_size=1, callbacks=[early_stopping])

Epoch 1/100
1/8 [==>...........................] - ETA: 4s - loss: 0.0000e+00 - accuracy: 0.0000e+00

InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\Users\Anon\.conda\envs\fin\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Anon\.conda\envs\fin\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\Anon\.conda\envs\fin\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Anon\.conda\envs\fin\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\Anon\.conda\envs\fin\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\Anon\.conda\envs\fin\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
   

In [173]:
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample dataset
questions = [
    {
        "js": [
            "How to install nodejs",
            "How do I install ts",
            "Why was ts created",
            "my javascript code is not working can someone help me"
        ]
    },
    {
        "python": [
            "How to create a python virtual environment",
            "my pip is not working python can someone help me",
            "how do I upgrade my pip python",
            "I want to update my python to version 2.4"
        ]
    }
]

# Preprocess the text using NLTK for tokenization and stopword removal
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return " ".join(filtered_tokens)

# Create lists to store the questions and corresponding labels
all_questions = []
all_labels = []

# Tokenize the text and convert to sequences
tokenizer = Tokenizer()

# Process each tag separately
for item in questions:
    for tag, tag_questions in item.items():
        # Preprocess all questions for the current tag
        preprocessed_questions = [preprocess_text(question) for question in tag_questions]
        
        # Append the preprocessed questions and corresponding labels
        all_questions.extend(preprocessed_questions)
        all_labels.extend([tag] * len(tag_questions))

# Fit the tokenizer on all questions
tokenizer.fit_on_texts(all_questions)
sequences = tokenizer.texts_to_sequences(all_questions)

# Pad sequences to ensure they have the same length
max_sequence_length = max(len(seq) for seq in sequences)
sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Convert labels to numerical labels
label_to_index = {tag: i for i, tag in enumerate(set(all_labels))}
labels = np.array([label_to_index[label] for label in all_labels], dtype=np.int32)

# Define the neural network architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(set(label_to_index)), activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Add Early Stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with Early Stopping
model.fit(sequences, labels, epochs=100, batch_size=1, validation_split=0.2, callbacks=[early_stopping])

# Now, you can use the model to predict the tag of a new question
new_question = "how is python"
new_question = preprocess_text(new_question)
new_sequence = tokenizer.texts_to_sequences([new_question])
new_padded_sequence = pad_sequences(new_sequence, maxlen=max_sequence_length)
print(new_padded_sequence)
# Predict probabilities for each class using the model's predict method
predicted_probs = model.predict(new_padded_sequence)

# Get the index of the predicted label with the highest probability
predicted_label_index = np.argmax(predicted_probs)

# Convert the index back to the original label
predicted_tag = [tag for tag, index in label_to_index.items() if index == predicted_label_index][0]

print("Predicted tag:", predicted_tag)


Epoch 1/100
6/6 [==============================] - 1s 55ms/step - loss: 0.6935 - accuracy: 0.6667 - val_loss: 0.7678 - val_accuracy: 0.0000e+00
Epoch 2/100
6/6 [==============================] - 0s 13ms/step - loss: 0.6142 - accuracy: 1.0000 - val_loss: 0.7929 - val_accuracy: 0.0000e+00
Epoch 3/100
6/6 [==============================] - 0s 12ms/step - loss: 0.5579 - accuracy: 1.0000 - val_loss: 0.8328 - val_accuracy: 0.0000e+00
Epoch 4/100
6/6 [==============================] - 0s 12ms/step - loss: 0.4964 - accuracy: 1.0000 - val_loss: 0.8764 - val_accuracy: 0.0000e+00
[[0 0 0 0 1]]
1/1 [==============================] - 0s 73ms/step
Predicted tag: js


In [167]:
new_question = "python"
new_question = preprocess_text(new_question)
new_sequence = tokenizer.texts_to_sequences([new_question])
new_padded_sequence = pad_sequences(new_sequence, maxlen=max_sequence_length)
predicted_probs = model.predict(new_padded_sequence)


predicted_label_index = np.argmax(predicted_probs)
predicted_tag = [tag for tag, index in label_to_index.items() if index == predicted_label_index][0]

print("Predicted tag:", predicted_tag)

1/1 [==============================] - 0s 35ms/step
Predicted tag: js
